# Installing Dependencies

# Import Libraries

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Load GPT-2 in 4-bit mode
model = AutoModelForCausalLM.from_pretrained(
    "gpt2",
    load_in_4bit=True,
    device_map="auto"  # Automatically places the model on the available GPUs
)

tokenizer = AutoTokenizer.from_pretrained("gpt2")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


# Import Model

In [4]:
# Prepare the model for 4-bit training (adds LoRA and disables weight decay on some params)
model = prepare_model_for_kbit_training(model)

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,               # Rank of the low-rank matrix -> A (input_dim, r) * B (r, output_dim)
    lora_alpha=32,     # LoRA scaling factor
    target_modules=["c_attn", "q_proj", "v_proj"],  # Layers to apply LoRA to (GPT-2 uses 'c_attn')
    lora_dropout=0.1,  # Dropout probability for LoRA layers
    bias="none",       # Bias configuration: can be "none", "all", or "lora_only"
    task_type="CAUSAL_LM"  # Task type for causal language modeling
)

# Add LoRA to the model
model = get_peft_model(model, lora_config)

# Importing and Processing Dataset

In [5]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./gpt2_qlora_python',
    overwrite_output_dir=True,
    num_train_epochs=3,  # Increase as needed; LoRA trains faster
    per_device_train_batch_size=4,  # You can increase this due to reduced memory usage
    gradient_accumulation_steps=16,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=500,
    logging_steps=100,
    logging_dir='./logs',
    save_total_limit=2,
    learning_rate=2e-4,  # Often higher LR works for QLoRA
    fp16=True,  # Enable mixed precision
    optim="paged_adamw_32bit",  # Optimizer for quantized training
    lr_scheduler_type="cosine",
)


c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
import pandas as pd
from datasets import Dataset
import torch
from transformers import AutoTokenizer

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and set the padding token
tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Replace with your model name
tokenizer.pad_token = tokenizer.eos_token

# Load your dataset from JSON
df_glaive = pd.read_json("hf://datasets/glaiveai/glaive-code-assistant/c9bc9129-eba0-4b10-8292-4ae70fc7fa0d.json")

# Convert the DataFrame to a HuggingFace Dataset
dataset = Dataset.from_pandas(df_glaive)

# Split the dataset into training and validation sets (90% train, 10% validation)
train_data = dataset.select([i for i in range(len(dataset)) if i % 10 != 0])  # 90% for training
val_data = dataset.select([i for i in range(len(dataset)) if i % 10 == 0])  # 10% for validation

# Tokenize the input and target sequences
def tokenize_function(examples):
    inputs = tokenizer(examples['question'], return_tensors='pt', padding='max_length', max_length=512, truncation=True)
    labels = tokenizer(examples['answer'], return_tensors='pt', padding='max_length', max_length=512, truncation=True)
    return {'input_ids': inputs['input_ids'], 'labels': labels['input_ids']}

# Apply tokenization to the datasets
train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

# Print a sample for debugging
print(train_data[0])
print(val_data[0])

Map:   0%|          | 0/122498 [00:00<?, ? examples/s]

Map:   0%|          | 0/13611 [00:00<?, ? examples/s]

{'question': "How can I install Python 3 on an AWS EC2 instance? I tried using the command `sudo yum install python3`, but I received an error message saying `No package python3 available.`. I searched online but didn't find a solution. Do I need to download and install it manually?", 'answer': "To install Python 3 on an AWS EC2 instance, you can use the Amazon Linux Extras Library. This library is a curated set of software that Amazon provides for the Amazon Linux 2 platform. It includes newer versions of software, like Python 3, that are not included in the default Amazon Linux 2 repositories. Here is a step by step process on how to do it:\n\n1. First, update your instance with the following command:\n\n```bash\nsudo yum update -y\n```\n\n2. Next, list available packages in the Amazon Linux Extras repository by typing:\n\n```bash\nsudo amazon-linux-extras list\n```\n\n3. You should see python3.8 available in the list. To install it, use the following command:\n\n```bash\nsudo amazon

# Train Model

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

trainer.train()

  0%|          | 0/5742 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\bitsandbytes\nn\modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter sh

{'loss': 5.8871, 'grad_norm': 1.0586110353469849, 'learning_rate': 0.00019985036436402656, 'epoch': 0.05}


  0%|          | 0/1702 [00:00<?, ?it/s]

{'eval_loss': 4.76005744934082, 'eval_runtime': 88.7801, 'eval_samples_per_second': 153.311, 'eval_steps_per_second': 19.171, 'epoch': 0.05}
{'loss': 4.6833, 'grad_norm': 1.3926193714141846, 'learning_rate': 0.00019940190527257726, 'epoch': 0.1}


...

  0%|          | 0/1702 [00:00<?, ?it/s]

{'eval_loss': 4.209779739379883, 'eval_runtime': 88.5965, 'eval_samples_per_second': 153.629, 'eval_steps_per_second': 19.211, 'epoch': 2.93}
{'loss': 4.2579, 'grad_norm': 0.8054270148277283, 'learning_rate': 2.6401149932875346e-08, 'epoch': 2.98}


  0%|          | 0/1702 [00:00<?, ?it/s]

{'eval_loss': 4.209802627563477, 'eval_runtime': 88.6813, 'eval_samples_per_second': 153.482, 'eval_steps_per_second': 19.192, 'epoch': 2.98}
{'train_runtime': 11854.7474, 'train_samples_per_second': 31.0, 'train_steps_per_second': 0.484, 'train_loss': 4.3554895388378325, 'epoch': 3.0}


TrainOutput(global_step=5742, training_loss=4.3554895388378325, metrics={'train_runtime': 11854.7474, 'train_samples_per_second': 31.0, 'train_steps_per_second': 0.484, 'total_flos': 9.635357044349338e+16, 'train_loss': 4.3554895388378325, 'epoch': 2.9999020408163264})

In [8]:
model.save_pretrained('./gpt2_qlora_finetuned_python')
tokenizer.save_pretrained('./gpt2_qlora_finetuned_python')

('./gpt2_qlora_finetuned_python\\tokenizer_config.json',
 './gpt2_qlora_finetuned_python\\special_tokens_map.json',
 './gpt2_qlora_finetuned_python\\vocab.json',
 './gpt2_qlora_finetuned_python\\merges.txt',
 './gpt2_qlora_finetuned_python\\added_tokens.json',
 './gpt2_qlora_finetuned_python\\tokenizer.json')

In [9]:
# Example input for pre-trained model
input_text = """Hi how are you?"""

# Tokenize the input
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate output
model.to(device)
pre_training_output = model.generate(
    input_ids,
    max_length=300,
    num_return_sequences=1,
    temperature=0.7,
    top_k=50,
)

# Decode and print the result
decoded_output = tokenizer.decode(pre_training_output[0], skip_special_tokens=True)
print("Output before training:")
print(decoded_output)

c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 w

Output before training:
Hi how are you? can can the you can the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
 the the the the the the the the the the the the the the the the the the

 the the the

 the
 the the the the the

 the the

 the the
 the the the the


 the
 the
 the
 the






 the

 the




 the


 the
 the the

 the the



 the
 the















 the

 the the






 the



 the

 the






 the





In [10]:
# Example input for pre-trained model
input_text = """what is the mistake in this python code here? a = [1,2,3,4,5,6)"""

# Tokenize the input
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate output
model.to(device)
pre_training_output = model.generate(
    input_ids,
    max_length=100,
    num_return_sequences=1,
    temperature=0.7,
    top_k=50,
)

# Decode and print the result
decoded_output = tokenizer.decode(pre_training_output[0], skip_special_tokens=True)
print("Output before training:")
print(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Output before training:
what is the mistake in this python code here? a = [1,2,3,4,5,6)
 error, error, error, error error, error, error error error error error
 error error error
 error error error error error error error error error
 error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error error


# Save Model

# Running Inference

In [11]:
# Inference Example
example_input = "<h1>Heading"
input_ids = tokenizer(example_input, return_tensors="pt")["input_ids"].to(device)
attention_mask = torch.ones(input_ids.shape, device=device)
output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1, top_k=50, top_p=0.95, attention_mask=attention_mask)

# Decode and print the corrected HTML code
decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Original HTML code:", example_input)
print("Corrected HTML code:", decoded_output)

c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Original HTML code: <h1>Heading
Corrected HTML code: <h1>Heading the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the

In [12]:
# Inference Example
example_input = """<p>This is a paragraph."""
input_ids = tokenizer(example_input, return_tensors="pt")["input_ids"].to(device)
attention_mask = torch.ones(input_ids.shape, device=device)
output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1, top_k=50, top_p=0.95, attention_mask=attention_mask)

# Decode and print the corrected HTML code
decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
#print("Original HTML code:", example_input)
print("Corrected HTML code:", decoded_output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Corrected HTML code: <p>This is a paragraph.

































In [13]:
# Inference Example
example_input = """<div><span>This is some text.</p></div>"""
input_ids = tokenizer(example_input, return_tensors="pt")["input_ids"].to(device)
attention_mask = torch.ones(input_ids.shape, device=device)
output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1, top_k=50, top_p=0.95, attention_mask=attention_mask)

# Decode and print the corrected HTML code
decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
#print("Original HTML code:", example_input)
print("Corrected HTML code:", decoded_output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Corrected HTML code: <div><span>This is some text.</p></div>><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><span><><span><span><span><span><span><span><span><span><span><span><span><span><span><span><><span><span><span><span><span><><span><span><span><><span><span><span><><span><><span><span><><span><><span><span><><span><span><span><><><><span><span><><><span><><span><span><><span><><span><span><><><><span><span><><span><><span><><><span><><><span><><><span><><><span><><><><span><><><><span><span><><><><><span><><><><><span><><><><><span><><><span><><><><><><span><><><><><span><><><><span><><><span><><><><><><><><><span><><><><><span><><><><><span><><><><><><><><span><><><><><><><span><><><><><><><><

In [14]:
# Inference Example
example_input = """list = [1,2,3,4,5,6"""
input_ids = tokenizer(example_input, return_tensors="pt")["input_ids"].to(device)
attention_mask = torch.ones(input_ids.shape, device=device)
output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1, top_k=10, top_p=0.97, attention_mask=attention_mask)

# Decode and print the corrected HTML code
decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
#print("Original HTML code:", example_input)
print("Corrected HTML code:", decoded_output)

c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.97` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\transformers\generation\configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Corrected HTML code: list = [1,2,3,4,5,6,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [21]:
# Inference Example
example_input = """<div>list<div>"""
input_ids = tokenizer(example_input, return_tensors="pt")["input_ids"].to(device)
attention_mask = torch.ones(input_ids.shape, device=device)
output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1, top_k=10, top_p=0.97, attention_mask=attention_mask)

# Decode and print the corrected HTML code
decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
#print("Original HTML code:", example_input)
print("Corrected HTML code:", decoded_output)

c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.97` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\transformers\generation\configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
c:\Users\kpreube\miniconda3\envs\llm\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need

Corrected HTML code: <div>list<div>div>divdiv>divdivdiv>div>div>div>div>divdiv>div>divdiv>div>div>div>div>div>div>div>div>div>div>div>div>div>div>div>div>div>div>div>div>div>div>div>div>div>divdiv>>div>div>div>div>div>div>divdiv>divdiv>div>divdiv>div>div>div>div>div>divdivdiv>divdiv>divdiv>divdiv>div>divdiv>div>divdiv>divdiv>div>div>divdivdivdiv>div>divdiv>div>div>div>div>divdiv>divdivdiv>divdivdivdiv>divdivdiv>divdivdiv>div>div>divdivdiv>divdivdiv>divdivdiv>divdivdivdivdiv>divdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdivdi